In [1]:
# Initial imports
import pandas as pd
from datetime import datetime
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [2]:
# Loading data
file_path = Path("202111-citibike-tripdata-cleaned.csv")
new_bikedata_df = pd.read_csv(file_path)
new_bikedata_df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
0,1,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member
1,2,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member
2,3,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member
3,4,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member
4,5,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member


In [3]:
#dt = '2015-05-13 23:53:00'
#date = dt.split()[0].replace('-','')
#time = dt.split()[1].replace(':','')
#fl = float(date+ '.' + time)

new_bikedata_df['started_at'] = pd.to_datetime(new_bikedata_df['started_at'])
new_bikedata_df['start_date'] = new_bikedata_df['started_at'].dt.date
new_bikedata_df['start_time'] = new_bikedata_df['started_at'].dt.time
new_bikedata_df['ended_at'] = pd.to_datetime(new_bikedata_df['ended_at'])
new_bikedata_df['end_date'] = new_bikedata_df['ended_at'].dt.date
new_bikedata_df['end_time'] = new_bikedata_df['ended_at'].dt.time

new_bikedata_df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,2021-11-08 07:34:45,2021-11-08 07:52:14,Franklin Ave & St Marks Ave,Carroll St & Smith St,40.675832,-73.956168,40.680611,-73.994758,member,2021-11-08,07:34:45,2021-11-08,07:52:14
1,2,1,2021-11-24 10:27:06,2021-11-24 10:34:40,Pleasant Ave & E 120 St,Willis Ave & E 137 St,40.797477,-73.931185,40.808384,-73.923604,member,2021-11-24,10:27:06,2021-11-24,10:34:40
2,3,1,2021-11-27 18:11:36,2021-11-27 19:19:16,West St & Liberty St,W 45 St & 8 Ave,40.711444,-74.014847,40.759291,-73.988597,member,2021-11-27,18:11:36,2021-11-27,19:19:16
3,4,1,2021-11-22 14:52:20,2021-11-22 15:30:40,W 67 St & Broadway,E 84 St & 1 Ave,40.774925,-73.982666,40.775655,-73.950686,member,2021-11-22,14:52:20,2021-11-22,15:30:40
4,5,1,2021-11-10 14:32:56,2021-11-10 14:36:19,W 53 St & 10 Ave,W 45 St & 8 Ave,40.766697,-73.990617,40.759291,-73.988597,member,2021-11-10,14:32:56,2021-11-10,14:36:19


In [4]:
new_bikedata_df.drop(["started_at", "ended_at", "start_station_name", "end_station_name"], inplace=True, axis=1)
new_bikedata_df.head()


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,member,2021-11-08,07:34:45,2021-11-08,07:52:14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,member,2021-11-24,10:27:06,2021-11-24,10:34:40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,member,2021-11-27,18:11:36,2021-11-27,19:19:16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,member,2021-11-22,14:52:20,2021-11-22,15:30:40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,member,2021-11-10,14:32:56,2021-11-10,14:36:19


In [5]:
new_bikedata_df.dtypes


ride_id            int64
rideable_type      int64
start_lat        float64
start_lng        float64
end_lat          float64
end_lng          float64
member_casual     object
start_date        object
start_time        object
end_date          object
end_time          object
dtype: object

In [6]:
# CHANGE THE DATE FORMAT FOR start_date
new_bikedata_df['start_date'] = new_bikedata_df['start_date'].apply(lambda x: pd.Timestamp(x).strftime('%Y%m%d'))
new_bikedata_df.head()


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,member,20211108,07:34:45,2021-11-08,07:52:14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,member,20211124,10:27:06,2021-11-24,10:34:40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,member,20211127,18:11:36,2021-11-27,19:19:16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,member,20211122,14:52:20,2021-11-22,15:30:40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,member,20211110,14:32:56,2021-11-10,14:36:19


In [7]:
# CHANGE THE DATE FORMAT FOR start_date
new_bikedata_df['end_date'] = new_bikedata_df['end_date'].apply(lambda x: pd.Timestamp(x).strftime('%Y%m%d'))
new_bikedata_df.head()


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,member,20211108,07:34:45,20211108,07:52:14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,member,20211124,10:27:06,20211124,10:34:40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,member,20211127,18:11:36,20211127,19:19:16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,member,20211122,14:52:20,20211122,15:30:40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,member,20211110,14:32:56,20211110,14:36:19


In [8]:
# TRANSFORM STRING COLUMN
def change_string(member_casual):
    if member_casual == "member":
        return 1
    else:
        return 0
    
new_bikedata_df["member_casual"] = new_bikedata_df["member_casual"].apply(change_string)
new_bikedata_df.head(20)


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,1,20211108,07:34:45,20211108,07:52:14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,1,20211124,10:27:06,20211124,10:34:40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,1,20211127,18:11:36,20211127,19:19:16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,1,20211122,14:52:20,20211122,15:30:40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,1,20211110,14:32:56,20211110,14:36:19
5,6,1,40.675832,-73.956168,40.673134,-73.969106,0,20211109,07:20:53,20211109,07:26:13
6,7,0,40.676520,-73.959608,40.673134,-73.969106,0,20211103,16:23:08,20211103,16:30:39
7,8,0,40.685930,-74.002424,40.686744,-73.990632,0,20211103,14:00:14,20211103,14:07:12
8,9,0,40.745712,-73.981948,40.759291,-73.988597,1,20211111,16:37:28,20211111,16:50:51
9,10,0,40.682800,-73.999904,40.680611,-73.994758,1,20211117,12:39:03,20211117,12:42:08


In [9]:
new_bikedata_df['start_time'] = new_bikedata_df['start_time'].astype(str).str.replace(':', ' ')
new_bikedata_df


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,1,20211108,07 34 45,20211108,07:52:14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,1,20211124,10 27 06,20211124,10:34:40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,1,20211127,18 11 36,20211127,19:19:16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,1,20211122,14 52 20,20211122,15:30:40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,1,20211110,14 32 56,20211110,14:36:19
...,...,...,...,...,...,...,...,...,...,...,...
2151489,-11198,0,40.771361,-73.924615,40.776700,-73.927631,1,20211121,11 59 01,20211121,12:02:54
2151490,-11197,0,40.825125,-73.941616,40.808442,-73.945209,1,20211112,09 29 41,20211112,09:40:20
2151491,-11196,0,40.714852,-74.011223,40.754557,-73.965930,1,20211118,17 36 02,20211118,18:10:08
2151492,-11195,1,40.821111,-73.935971,40.808442,-73.945209,0,20211113,12 06 53,20211113,12:14:42


In [10]:
new_bikedata_df['end_time'] = new_bikedata_df['end_time'].astype(str).str.replace(':', ' ')
new_bikedata_df


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,1,1,40.675832,-73.956168,40.680611,-73.994758,1,20211108,07 34 45,20211108,07 52 14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,1,20211124,10 27 06,20211124,10 34 40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,1,20211127,18 11 36,20211127,19 19 16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,1,20211122,14 52 20,20211122,15 30 40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,1,20211110,14 32 56,20211110,14 36 19
...,...,...,...,...,...,...,...,...,...,...,...
2151489,-11198,0,40.771361,-73.924615,40.776700,-73.927631,1,20211121,11 59 01,20211121,12 02 54
2151490,-11197,0,40.825125,-73.941616,40.808442,-73.945209,1,20211112,09 29 41,20211112,09 40 20
2151491,-11196,0,40.714852,-74.011223,40.754557,-73.965930,1,20211118,17 36 02,20211118,18 10 08
2151492,-11195,1,40.821111,-73.935971,40.808442,-73.945209,0,20211113,12 06 53,20211113,12 14 42


In [11]:
new_bikedata_df[['start_hours', 'start_minutes', 'start_seconds']] = new_bikedata_df['start_time'].str.split(' ', expand=True)
new_bikedata_df[['end_hours', 'end_minutes', 'end_seconds']] = new_bikedata_df['end_time'].str.split(' ', expand=True)


In [12]:
new_bikedata_df.drop(["start_time", "end_time"], inplace=True, axis=1)
new_bikedata_df.head()


,ride_id,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,end_date,start_hours,start_minutes,start_seconds,end_hours,end_minutes,end_seconds
0,1,1,40.675832,-73.956168,40.680611,-73.994758,1,20211108,20211108,07,34,45,07,52,14
1,2,1,40.797477,-73.931185,40.808384,-73.923604,1,20211124,20211124,10,27,06,10,34,40
2,3,1,40.711444,-74.014847,40.759291,-73.988597,1,20211127,20211127,18,11,36,19,19,16
3,4,1,40.774925,-73.982666,40.775655,-73.950686,1,20211122,20211122,14,52,20,15,30,40
4,5,1,40.766697,-73.990617,40.759291,-73.988597,1,20211110,20211110,14,32,56,14,36,19


In [13]:
y = new_bikedata_df['rideable_type']
X = new_bikedata_df['member_casual']


In [14]:
# SPLIT THE DATASET
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)


In [15]:
# CREATE LOGISTIC REGRESSION MODEL
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)


In [16]:
classifier.fit(X_train, y_train)


ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 1. ... 1. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
y_pred = classifier.predict(X_test)
